In [1]:
from astropy.table import Table, Column, table, vstack
from pathlib import Path

import numpy as np
import astropy.units as u

from matplotlib import pyplot as plt

In [2]:
path_raw='../Datos/Raw/'
path_phot_ned = '../Datos/Work/Phot_Tables_NED/' #In this path wi will save the photometry tables for each object
path_phot_cds = '../Datos/Work/Phot_Tables_CDS/'
path_SED = '../Datos/Work/SEDs/'
path_CIGALE_phot = '../Datos/Work/CIGALE_InputPhot/'
path_CIGALE_out = '../Datos/Work/CIGALE_Output/'
path_FINAL = '../Datos/Final/'

In [3]:
date_sample = 'Sep_02_2022'
NeV_NeII_Sy = Table.read(path_raw+'Obj_Sy_Ne-IR_'+date_sample+'.vot',format='votable')
NeV_NeIII_Sy = Table.read(path_raw+'Obj_Sy_Ne-IR_'+date_sample+'.vot',format='votable')
NeIII_NeII_Sy = Table.read(path_raw+'Obj_Sy_Ne-IR_'+date_sample+'.vot',format='votable')

In [4]:
#Rows to be deleted or removed in each table
to_rem1 = [] 
to_rem2 = []
to_rem3 = []

for i in range(len(NeV_NeII_Sy)): 
    if (NeV_NeII_Sy['NED_Ne_IR_info'][i] != 'NeII|NeIII|NeV' and NeV_NeII_Sy['NED_Ne_IR_info'][i]!= 'NeII|NeV'):
        to_rem1.append(i)
        
for i in range(len(NeV_NeIII_Sy)): 
    if (NeV_NeIII_Sy['NED_Ne_IR_info'][i] != 'NeII|NeIII|NeV' and NeV_NeIII_Sy['NED_Ne_IR_info'][i]!= 'NeIII|NeV'):
        to_rem2.append(i)

for i in range(len(NeIII_NeII_Sy)): 
    if (NeV_NeII_Sy['NED_Ne_IR_info'][i] != 'NeII|NeIII|NeV' and NeV_NeII_Sy['NED_Ne_IR_info'][i]!= 'NeII|NeIII'):
        to_rem3.append(i)
    
NeV_NeII_Sy.remove_rows(to_rem1)
NeV_NeIII_Sy.remove_rows(to_rem2)
NeIII_NeII_Sy.remove_rows(to_rem3)
#NeII_NeV_Sy.show_in_notebook()
print('Total objects with [NeV] and [NeII] info:',len(NeV_NeII_Sy))
print('Total objects with [NeV] and [NeIII] info:',len(NeV_NeIII_Sy))
print('Total objects with [NeIII] and [NeII] info:',len(NeIII_NeII_Sy))

Total objects with [NeV] and [NeII] info: 183
Total objects with [NeV] and [NeIII] info: 181
Total objects with [NeIII] and [NeII] info: 227


In [5]:
def Neon_info(NED_table, Freq):
    
    '''
    This function create a new table from the Phot NED table
    but with the flux of [NeII] and [NeV]
    '''
    
    NED_table.remove_rows(np.where(NED_table['Flux_Density'].mask)[0])
    NED_table.remove_rows(np.where(NED_table['NED_Units'] == 'Jy')[0])
    NED_table.remove_rows(np.where(NED_table['Observed_Passband'] == '[Cl II] line (IRS)')[0])
    NED_table.remove_rows(np.where(NED_table['Observed_Passband'] == '[Cl II] 14.4 (IRS)')[0])
    NED_table.remove_rows(np.where(NED_table['Observed_Passband'] == '[Cl II] (IRS)')[0])
    NED_table.remove_rows(np.where(NED_table['Observed_Passband'] == '[Cl II] 14.37 (IRS)')[0])
    
    Ne_idx = [np.where(NED_table['Frequency'] == freq)[0] for freq in Freq]
    #print(Ne_idx)
    #print(len(Ne_idx))
    i = []
    Ne_table = Table()
    for idx in range(len(Freq)):
        Ne_table = vstack([Ne_table,NED_table[Ne_idx[idx]]])
    
    for ind in range(len(Ne_table)):
        i.append(ind)
    
    Ne_table['idx'] = i
    Ne_inf = Ne_table['idx','Observed_Passband','Flux_Density','NED_Uncertainty','NED_Units','Refcode']
    obs_passb = np.unique(Ne_inf['Observed_Passband'])
    #print(obs_passb)
    return Ne_inf

In [25]:
def Ne_ratios(Ne_info,Nel1,Nel2,Nel1a,Nel2a,passband,refcode,not_same_bib):
    bibcod, count = np.unique(Ne_info['Refcode'], return_counts=True)
    print('No. of [Ne] data',len(Ne_info))
    flag = 0 #Flag to detect when there is not info of the two Ne lines for the same bibcode
    print('Neon line 1:',Nel1,'& Neon line 2:',Nel2)
    print('Others filter names for Neon line1:',Nel1a,'& Neon line 2:',Nel2a)
    Ne_ratio = []
    
    if len(Ne_info)>2:
        print(Ne_info)
        for idx,bib in enumerate(bibcod):
            i = np.where(Ne_info['Refcode'] == bib)[0]
            #print(i)
            #The ratio [Ne]line1/[Ne]line2 is calculated for only the bibcodes that have both flux data
            if len(i)>=2:
                flag += 1
                #print(len(i))
                Neline1_i = 'nan'
                Neline2_i = 'nan'
                for j,ind in enumerate(i):
                    if Nel1 in Ne_info['Observed_Passband'][ind] or Nel1a in Ne_info['Observed_Passband'][ind]:
                        #print('yes:',Nel1)
                        Neline1_i = ind
                    elif Nel2 in Ne_info['Observed_Passband'][ind] or Nel2a in Ne_info['Observed_Passband'][ind]:
                        #print('yes:',Nel2)
                        Neline2_i = ind
                #If of both data per bibcode, one of [Ne]lines is misssing then the row is removed
                #if Neline1_i == 'nan' or Neline2_i == 'nan':
                    #Ne_info.remove_rows(np.where(Ne_info['Refcode'] == bib)[0])
                if Neline1_i != 'nan' and Neline2_i != 'nan':
                    Neline1 = Ne_info['Flux_Density'][Neline1_i]
                    Neline2 = Ne_info['Flux_Density'][Neline2_i]
                    Neline1_Neline2_ratio = Neline1/Neline2
                    Ne_ratio.append(Neline1_Neline2_ratio)
                    passband.append(Ne_info['Observed_Passband'][Neline1_i])
                    passband.append(Ne_info['Observed_Passband'][Neline2_i])
                    refcode.append(bib)
                    print('For bibcode',',',bib,Nel1,'index is:',Neline1_i,'and',\
                          Nel2,'index is:',Neline2_i)
                    print('For bibcode', bib,Nel1,'/',Nel2,'ratio is:',Neline1_Neline2_ratio)
        if flag<1:
            flag = 0
            not_same_bib += 1
            #for r in range(len(NeV_NeII_inf)): refcode.append(NeV_NeII_inf['Refcode'][r])
    elif len(Ne_info)==2:
        print(Ne_info)
        Neline1 = Ne_info['Flux_Density'][0]
        Neline2 = Ne_info['Flux_Density'][1]
        Neline1_Neline2_ratio = Neline1/Neline2
        Ne_ratio.append(Neline1_Neline2_ratio)
        if Ne_info['Refcode'][0] == Ne_info['Refcode'][1]:
            passband.append(Ne_info['Observed_Passband'][0])
            refcode.append(Ne_info['Refcode'][0])
            print('For bibcode',Ne_info['Refcode'][0],',',Nel1,'index is: 0 and',Nel2,'index is: 1')
            print('For bibcode', Ne_info['Refcode'][0],Nel1,'/',Nel2,'ratio is:',Neline1_Neline2_ratio)
        elif Ne_info['Refcode'][0] != Ne_info['Refcode'][1]:
            #passband.append(NeV_NeII_inf['Observed_Passband'][0])
            #passband.append(NeV_NeII_inf['Observed_Passband'][1])
            #refcode.append(Ne_info['Refcode'][0])
            #refcode.append(Ne_info['Refcode'][1])
            not_same_bib += 1
            
    return Ne_ratio, passband, refcode, not_same_bib

In the galaxies where there are more than one data of [NeII] or [NeV], we remove the rows if per bibcode there is data for only one the them. The idea is to have both [NeII] and [NeV] data per bibcode:

## [NeV]/[NeII]

In [34]:
passband = []
refcode = []
NeV_NeII = Table(names=('Main_id','[NeV]/[NeII]','[NeV]/[NeII]_err'))

not_same_bib = 0

for id in range(len(NeV_NeII_Sy)):
    NED_phot = Table()
    obj = NeV_NeII_Sy['Main_Id'][id]
    print(id)
    print('Galaxy: ',obj)

    NED_phot = Table.read(path_phot_ned+obj+'_NED_phot_tables'+'.vot', format='votable')
    print('Phot table lenght:',len(NED_phot))
    
    NeV_NeII_Freq = [20900000000000.0,21000000000000.0,23400000000000.0]
    NeV_NeII_inf = Neon_info(NED_phot, NeV_NeII_Freq)
    
    NeV_NeII_ratio, passband,refcode,not_same_bib = Ne_ratios(NeV_NeII_inf,'[Ne V]','[Ne II]',\
                                                              'no name','no name',passband,refcode,not_same_bib)
    
    #print(NeV_NeII_ratio)
    print('\n') 

0
Galaxy:  3C 234.0
Phot table lenght: 253
No. of [Ne] data 4
Neon line 1: [Ne V] & Neon line 2: [Ne II]
Others filter names for Neon line1: no name & Neon line 2: no name
idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- -------------------- ------------ ... --------- -------------------
  0   [Ne V] 14.32 (IRS) 2350000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  1  [Ne V] 14.3 microns 3300000000.0 ...     Jy-Hz 2006ApJ...647..161O
  2   [Ne II] 12.8 (IRS) 1400000000.0 ...     Jy-Hz 2009ApJ...697..182T
  3 [Ne II] 12.81 micron  800000000.0 ...     Jy-Hz 2006ApJ...647..161O
For bibcode , 2006ApJ...647..161O [Ne V] index is: 1 and [Ne II] index is: 3
For bibcode 2006ApJ...647..161O [Ne V] / [Ne II] ratio is: 4.125


1
Galaxy:  NGC   262
Phot table lenght: 334
No. of [Ne] data 7
Neon line 1: [Ne V] & Neon line 2: [Ne II]
Others filter names for Neon line1: no name & Neon line 2: no name
idx  Ob

Phot table lenght: 198
No. of [Ne] data 9
Neon line 1: [Ne V] & Neon line 2: [Ne II]
Others filter names for Neon line1: no name & Neon line 2: no name
idx  Observed_Passband    Flux_Density ... NED_Units       Refcode      
                               Jy      ...                              
--- -------------------- ------------- ... --------- -------------------
  0   [Ne V] 14.32 (IRS)  8210000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  1 [Ne V] 14.32 Spitzer  9000000000.0 ...     Jy-Hz 2007ApJ...671..124D
  2  [Ne V] 14.3 Spitzer  7200000000.0 ...     Jy-Hz 2010ApJ...725.2270P
  3         [Ne V] 14.32  8210000000.0 ...     Jy-Hz 2011ApJS..195...17W
  4    [Ne V] 14.3 (IRS)  8210000000.0 ...     Jy-Hz 2009ApJ...690.1105K
  5        [Ne II] 12.81 13500000000.0 ...     Jy-Hz 2011ApJS..195...17W
  6 [Ne II] 12.8 Spitzer 13000000000.0 ...     Jy-Hz 2010ApJ...725.2270P
  7  [Ne II] 12.81 (IRS) 13500000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  8   [Ne II] 12.8 (IRS) 14700000000.0 ...   

No. of [Ne] data 6
Neon line 1: [Ne V] & Neon line 2: [Ne II]
Others filter names for Neon line1: no name & Neon line 2: no name
idx  Observed_Passband    Flux_Density ... NED_Units       Refcode      
                               Jy      ...                              
--- -------------------- ------------- ... --------- -------------------
  0 [Ne V] 14.32 Spitzer  9000000000.0 ...     Jy-Hz 2007ApJ...671..124D
  1  [Ne V] 14.3 Spitzer  9000000000.0 ...     Jy-Hz 2010ApJ...725.2270P
  2         [Ne V] 14.32 11200000000.0 ...     Jy-Hz 2011ApJS..195...17W
  3        [Ne II] 12.81 45200000000.0 ...     Jy-Hz 2011ApJS..195...17W
  4 [Ne II] 12.8 Spitzer 42000000000.0 ...     Jy-Hz 2010ApJ...725.2270P
  5    [Ne II] (Spitzer) 39000000000.0 ...     Jy-Hz 2007ApJ...671..124D
For bibcode , 2007ApJ...671..124D [Ne V] index is: 0 and [Ne II] index is: 5
For bibcode 2007ApJ...671..124D [Ne V] / [Ne II] ratio is: 0.23076923076923078
For bibcode , 2010ApJ...725.2270P [Ne V] index is: 1 and [

Phot table lenght: 148
No. of [Ne] data 4
Neon line 1: [Ne V] & Neon line 2: [Ne II]
Others filter names for Neon line1: no name & Neon line 2: no name
idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- ------------------- ------------- ... --------- -------------------
  0        [Ne V] 14.32  2560000000.0 ...     Jy-Hz 2011ApJS..195...17W
  1        [Ne V] 14.32  2200000000.0 ...     Jy-Hz 2007ApJ...664...71D
  2 [Ne V] 14.3 Spitzer  2040000000.0 ...     Jy-Hz 2011ApJ...730...28P
  3       [Ne II] 12.81 27300000000.0 ...     Jy-Hz 2011ApJS..195...17W
For bibcode , 2011ApJS..195...17W [Ne V] index is: 0 and [Ne II] index is: 3
For bibcode 2011ApJS..195...17W [Ne V] / [Ne II] ratio is: 0.09377289377289377


31
Galaxy:  Mrk 1383
Phot table lenght: 205
No. of [Ne] data 7
Neon line 1: [Ne V] & Neon line 2: [Ne II]
Others filter names for Neon line1: no name & Neon line 2: no name
idx  Observed

No. of [Ne] data 3
Neon line 1: [Ne V] & Neon line 2: [Ne II]
Others filter names for Neon line1: no name & Neon line 2: no name
idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- -------------------- ------------ ... --------- -------------------
  0 [Ne V] 14.32 Spitzer 2000000000.0 ...     Jy-Hz 2007ApJ...671..124D
  1 [Ne II] 12.8 Spitzer 3200000000.0 ...     Jy-Hz 2010ApJ...725..605S
  2    [Ne II] (Spitzer) 3000000000.0 ...     Jy-Hz 2007ApJ...671..124D
For bibcode , 2007ApJ...671..124D [Ne V] index is: 0 and [Ne II] index is: 2
For bibcode 2007ApJ...671..124D [Ne V] / [Ne II] ratio is: 0.6666666666666666


42
Galaxy:  NGC  7679
Phot table lenght: 261
No. of [Ne] data 2
Neon line 1: [Ne V] & Neon line 2: [Ne II]
Others filter names for Neon line1: no name & Neon line 2: no name
idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy    

...

No. of [Ne] data 8
Neon line 1: [Ne V] & Neon line 2: [Ne II]
Others filter names for Neon line1: no name & Neon line 2: no name
idx  Observed_Passband   Flux_Density  ... NED_Units       Refcode      
                              Jy       ...                              
--- ------------------- -------------- ... --------- -------------------
  0  [Ne V] 14.32 (IRS)   9020000000.0 ...     Jy-Hz 2011ApJ...740...94D
  1        [Ne V] 14.32   5100000000.0 ...     Jy-Hz 2007ApJ...664...71D
  2        [Ne V] (IRS)   5100000000.0 ...     Jy-Hz 2006ApJ...640..204A
  3 [Ne V] 14.3 Spitzer   2070000000.0 ...     Jy-Hz 2011ApJ...730...28P
  4 [Ne II] 12.81 (IRS) 187000000000.0 ...     Jy-Hz 2011ApJ...740...94D
  5       [Ne II] (IRS) 193000000000.0 ...     Jy-Hz 2006ApJ...640..204A
  6    [Ne II] (TIMMI2)  50000000000.0 ...     Jy-Hz 2004A&A...414..123S
  7       [Ne II] (ISO) 170000000000.0 ...     Jy-Hz 2004A&A...414..123S
For bibcode , 2006ApJ...640..204A [Ne V] index is: 2 and [Ne II] ind

Phot table lenght: 425
No. of [Ne] data 16
Neon line 1: [Ne V] & Neon line 2: [Ne II]
Others filter names for Neon line1: no name & Neon line 2: no name
idx  Observed_Passband    Flux_Density ... NED_Units       Refcode      
                               Jy      ...                              
--- -------------------- ------------- ... --------- -------------------
  0   [Ne V] 14.32 (IRS) 10800000000.0 ...     Jy-Hz 2011ApJ...740...94D
  1    [Ne V] 14.3 (IRS) 11300000000.0 ...     Jy-Hz 2009MNRAS.398.1165G
  2 [Ne V] 14.32 Spitzer 11100000000.0 ...     Jy-Hz 2008ApJ...674L...9D
  3   [Ne V] 14.32 (IRS) 10200000000.0 ...     Jy-Hz 2010ApJ...716.1151W
  4         [Ne V] 14.32 10700000000.0 ...     Jy-Hz 2011ApJS..195...17W
  5  [Ne V] 14.3 Spitzer 11000000000.0 ...     Jy-Hz 2010ApJ...725.2270P
  6 [Ne V] 14.32 Spitzer 18000000000.0 ...     Jy-Hz 2007ApJ...671..124D
  7   [Ne V] 14.32 (IRS) 10700000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  8        [Ne II] 12.81 21200000000.0 ...  

No. of [Ne] data 7
Neon line 1: [Ne V] & Neon line 2: [Ne II]
Others filter names for Neon line1: no name & Neon line 2: no name
idx  Observed_Passband    Flux_Density ... NED_Units       Refcode      
                               Jy      ...                              
--- -------------------- ------------- ... --------- -------------------
  0   [Ne V] 14.32 (IRS)  1140000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  1         [Ne V] 14.32  1140000000.0 ...     Jy-Hz 2011ApJS..195...17W
  2 [Ne V] 14.32 Spitzer  4000000000.0 ...     Jy-Hz 2007ApJ...671..124D
  3  [Ne II] 12.81 (IRS) 13200000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  4        [Ne II] 12.81 13200000000.0 ...     Jy-Hz 2011ApJS..195...17W
  5  [Ne II] 12.81 (IRS) 10500000000.0 ...     Jy-Hz 2011ApJ...740...94D
  6    [Ne II] (Spitzer) 21000000000.0 ...     Jy-Hz 2007ApJ...671..124D
For bibcode , 2007ApJ...671..124D [Ne V] index is: 2 and [Ne II] index is: 6
For bibcode 2007ApJ...671..124D [Ne V] / [Ne II] ratio is: 0.190

Phot table lenght: 107
No. of [Ne] data 2
Neon line 1: [Ne V] & Neon line 2: [Ne II]
Others filter names for Neon line1: no name & Neon line 2: no name
idx  Observed_Passband  Flux_Density ... NED_Units       Refcode      
                             Jy      ...                              
--- ------------------- ------------ ... --------- -------------------
  0  [Ne V] 14.32 (IRS) 1640000000.0 ...     Jy-Hz 2011ApJ...740...94D
  1 [Ne II] 12.81 (IRS) 2490000000.0 ...     Jy-Hz 2011ApJ...740...94D
For bibcode 2011ApJ...740...94D , [Ne V] index is: 0 and [Ne II] index is: 1
For bibcode 2011ApJ...740...94D [Ne V] / [Ne II] ratio is: 0.6586345381526104


172
Galaxy:  Mrk  841
Phot table lenght: 206
No. of [Ne] data 7
Neon line 1: [Ne V] & Neon line 2: [Ne II]
Others filter names for Neon line1: no name & Neon line 2: no name
idx  Observed_Passband  Flux_Density ... NED_Units       Refcode      
                             Jy      ...                              
--- ----------------

No. of [Ne] data 7
Neon line 1: [Ne V] & Neon line 2: [Ne II]
Others filter names for Neon line1: no name & Neon line 2: no name
idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- ------------------- ------------- ... --------- -------------------
  0        [Ne V] 14.32 21000000000.0 ...     Jy-Hz 2011ApJS..195...17W
  1  [Ne V] 14.32 (IRS) 21000000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  2  [Ne V] 14.32 (IRS) 22100000000.0 ...     Jy-Hz 2010ApJ...716.1151W
  3       [Ne II] 12.81 23300000000.0 ...     Jy-Hz 2011ApJS..195...17W
  4 [Ne II] 12.81 (IRS) 24300000000.0 ...     Jy-Hz 2010ApJ...716.1151W
  5 [Ne II] 12.81 (IRS) 23300000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  6  [Ne II] 12.8 (IRS) 33000000000.0 ...     Jy-Hz 2009ApJ...697..182T
For bibcode , 2010ApJ...709.1257T [Ne V] index is: 1 and [Ne II] index is: 5
For bibcode 2010ApJ...709.1257T [Ne V] / [Ne II] ratio is: 0.9012875536480

In [27]:
passbands, countspb = np.unique(passband, return_counts=True)
for pb in range(len(passbands)): print(passbands[pb])

[Ne II] (IRS)
[Ne II] (Spitzer)
[Ne II] 12.8 (IRS)
[Ne II] 12.8 Spitzer
[Ne II] 12.81
[Ne II] 12.81 (IRS)
[Ne II] 12.81 micron
[Ne II] line (IRS)
[Ne V] (IRS)
[Ne V] 14.3 (IRS)
[Ne V] 14.3 Spitzer
[Ne V] 14.3 microns
[Ne V] 14.32
[Ne V] 14.32 (IRS)
[Ne V] 14.32 Spitzer
[Ne V] line (IRS)


In [28]:
refcodes, countsrc = np.unique(refcode, return_counts=True)
for rc in range(len(refcodes)): print(refcodes[rc])

2004ApJS..154..178A
2006ApJ...640..204A
2006ApJ...647..161O
2006ApJ...649...79S
2007ApJ...656..148A
2007ApJ...660..117C
2007ApJ...671..124D
2009ApJ...690.1105K
2009MNRAS.398.1165G
2010ApJ...709.1257T
2010ApJ...716.1151W
2010ApJ...725.2270P
2011A&A...533A..56P
2011ApJ...740...94D
2011ApJ...740...99D
2011ApJS..193...18W
2011ApJS..195...17W
2012ApJ...747...46P
2012ApJ...747...95G


In [10]:
print('Number of galaxies with diferent bibcode for [NeV] and [NeII] fluxes:',not_same_bib)

Number of galaxies with diferent bibcode for [NeV] and [NeII] fluxes: 5


## [NeV]/[NeIII]

In [31]:
passband = []
refcode = []
NeV_NeIII = Table(names=('Main_id','[NeV]/[NeIII]','[NeV]/[NeIII]_err'))

not_same_bib = 0

for id in range(len(NeV_NeIII_Sy)):
    NED_phot = Table()
    obj = NeV_NeIII_Sy['Main_Id'][id]
    print(id)
    print('Galaxy: ',obj)

    NED_phot = Table.read(path_phot_ned+obj+'_NED_phot_tables'+'.vot', format='votable')
    print('Phot table lenght:',len(NED_phot))
    
    #Put first the frequencies of the Ne line 1 and then the Ne line 2
    NeV_NeIII_Freq = [20900000000000.0,21000000000000.0,19200000000000.0,19300000000000.0]
    NeV_NeIII_inf = Neon_info(NED_phot, NeV_NeIII_Freq)
    
    NeV_NeIII_ratio,passband,refcode,not_same_bib = Ne_ratios(NeV_NeIII_inf,'[Ne V]','[Ne III]',\
                                                             'no name','[NeIII]',passband,refcode,not_same_bib)
    
    print(NeV_NeIII_ratio)
    print('\n')

0
Galaxy:  3C 234.0
Phot table lenght: 253
No. of [Ne] data 4
Neon line 1: [Ne V] & Neon line 2: [Ne III]
Others filter names for Neon line1: no name & Neon line 2: [NeIII]
idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- -------------------- ------------ ... --------- -------------------
  0   [Ne V] 14.32 (IRS) 2350000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  1  [Ne V] 14.3 microns 3300000000.0 ...     Jy-Hz 2006ApJ...647..161O
  2 [Ne III] 15.56 (IRS) 3400000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  3 [Ne III] 15.55 micrn 8200000000.0 ...     Jy-Hz 2006ApJ...647..161O
For bibcode , 2006ApJ...647..161O [Ne V] index is: 1 and [Ne III] index is: 3
For bibcode 2006ApJ...647..161O [Ne V] / [Ne III] ratio is: 0.4024390243902439
For bibcode , 2010ApJ...709.1257T [Ne V] index is: 0 and [Ne III] index is: 2
For bibcode 2010ApJ...709.1257T [Ne V] / [Ne III] ratio is: 0.6911764705882353
[0.402439

No. of [Ne] data 2
Neon line 1: [Ne V] & Neon line 2: [Ne III]
Others filter names for Neon line1: no name & Neon line 2: [NeIII]
idx Observed_Passband Flux_Density NED_Uncertainty NED_Units       Refcode      
                           Jy                                                   
--- ----------------- ------------ --------------- --------- -------------------
  0      [Ne V] 14.32 5780000000.0                     Jy-Hz 2011ApJS..195...17W
  1    [Ne III] 15.56 8130000000.0                     Jy-Hz 2011ApJS..195...17W
For bibcode 2011ApJS..195...17W , [Ne V] index is: 0 and [Ne III] index is: 1
For bibcode 2011ApJS..195...17W [Ne V] / [Ne III] ratio is: 0.7109471094710947
[0.7109471094710947]


9
Galaxy:  Z 218-7
Phot table lenght: 87
No. of [Ne] data 2
Neon line 1: [Ne V] & Neon line 2: [Ne III]
Others filter names for Neon line1: no name & Neon line 2: [NeIII]
idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ... 

No. of [Ne] data 4
Neon line 1: [Ne V] & Neon line 2: [Ne III]
Others filter names for Neon line1: no name & Neon line 2: [NeIII]
idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- -------------------- ------------ ... --------- -------------------
  0   [Ne V] 14.32 (IRS)  957000000.0 ...     Jy-Hz 2011ApJ...740...94D
  1         [Ne V] 14.32 1010000000.0 ...     Jy-Hz 2011ApJS..195...17W
  2 [Ne V] 14.32 Spitzer 1010000000.0 ...     Jy-Hz 2008ApJ...674L...9D
  3 [Ne III] 15.56 (IRS) 2110000000.0 ...     Jy-Hz 2011ApJ...740...94D
For bibcode , 2011ApJ...740...94D [Ne V] index is: 0 and [Ne III] index is: 3
For bibcode 2011ApJ...740...94D [Ne V] / [Ne III] ratio is: 0.45355450236966827
[0.45355450236966827]


19
Galaxy:  IRAS 11215-2806
Phot table lenght: 19
No. of [Ne] data 2
Neon line 1: [Ne V] & Neon line 2: [Ne III]
Others filter names for Neon line1: no name & Neon line 2: [NeIII]
idx 

No. of [Ne] data 2
Neon line 1: [Ne V] & Neon line 2: [Ne III]
Others filter names for Neon line1: no name & Neon line 2: [NeIII]
idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- -------------------- ------------ ... --------- -------------------
  0   [Ne V] 14.32 (IRS) 1160000000.0 ...     Jy-Hz 2010ApJ...716.1151W
  1 [Ne III] 15.56 (IRS) 1380000000.0 ...     Jy-Hz 2010ApJ...716.1151W
For bibcode 2010ApJ...716.1151W , [Ne V] index is: 0 and [Ne III] index is: 1
For bibcode 2010ApJ...716.1151W [Ne V] / [Ne III] ratio is: 0.8405797101449275
[0.8405797101449275]


28
Galaxy:  NGC  1142
Phot table lenght: 257
No. of [Ne] data 4
Neon line 1: [Ne V] & Neon line 2: [Ne III]
Others filter names for Neon line1: no name & Neon line 2: [NeIII]
idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- -------

No. of [Ne] data 6
Neon line 1: [Ne V] & Neon line 2: [Ne III]
Others filter names for Neon line1: no name & Neon line 2: [NeIII]
idx  Observed_Passband    Flux_Density ... NED_Units       Refcode      
                               Jy      ...                              
--- -------------------- ------------- ... --------- -------------------
  0  [Ne V] 14.3 microns 12400000000.0 ...     Jy-Hz 2007ApJ...655L..73G
  1    [Ne V] 14.3 (IRS) 12200000000.0 ...     Jy-Hz 2009ApJ...690.1105K
  2  [Ne V] 14.3 Spitzer 12000000000.0 ...     Jy-Hz 2010ApJ...725.2270P
  3 [NeIII] 15.6 microns 25500000000.0 ...     Jy-Hz 2007ApJ...655L..73G
  4  [Ne III] 15.5 (IRS) 25300000000.0 ...     Jy-Hz 2009ApJ...690.1105K
  5 [NeIII] 15.6 Spitzer 24000000000.0 ...     Jy-Hz 2010ApJ...725.2270P
For bibcode , 2007ApJ...655L..73G [Ne V] index is: 0 and [Ne III] index is: 3
For bibcode 2007ApJ...655L..73G [Ne V] / [Ne III] ratio is: 0.48627450980392156
For bibcode , 2009ApJ...690.1105K [Ne V] index is: 1 an

...

No. of [Ne] data 10
Neon line 1: [Ne V] & Neon line 2: [Ne III]
Others filter names for Neon line1: no name & Neon line 2: [NeIII]
idx  Observed_Passband    Flux_Density  ... NED_Units       Refcode      
                               Jy       ...                              
--- -------------------- -------------- ... --------- -------------------
  0   [Ne V] 14.32 (IRS)  63300000000.0 ...     Jy-Hz 2011ApJ...740...94D
  1 [Ne V] 14.32 Spitzer 109000000000.0 ...     Jy-Hz 2007ApJ...671..124D
  2         [Ne V] 14.32 109000000000.0 ...     Jy-Hz 2011ApJS..195...17W
  3         [Ne V] 14.32  64500000000.0 ...     Jy-Hz 2007ApJ...664...71D
  4    [Ne V] 14.3 (IRS)  64600000000.0 ...     Jy-Hz 2009ApJ...690.1105K
  5  [Ne III] 15 Spitzer 209000000000.0 ...     Jy-Hz 2010ApJ...725..605S
  6 [Ne III] 15.56 (IRS) 175000000000.0 ...     Jy-Hz 2011ApJ...740...94D
  7       [Ne III] 15.56 207000000000.0 ...     Jy-Hz 2011ApJS..195...17W
  8   [Ne III] (Spitzer) 207000000000.0 ...     Jy-Hz 2

No. of [Ne] data 2
Neon line 1: [Ne V] & Neon line 2: [Ne III]
Others filter names for Neon line1: no name & Neon line 2: [NeIII]
idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- -------------------- ------------ ... --------- -------------------
  0   [Ne V] 14.32 (IRS) 2900000000.0 ...     Jy-Hz 2010ApJ...716.1151W
  1 [Ne III] 15.56 (IRS) 6070000000.0 ...     Jy-Hz 2010ApJ...716.1151W
For bibcode 2010ApJ...716.1151W , [Ne V] index is: 0 and [Ne III] index is: 1
For bibcode 2010ApJ...716.1151W [Ne V] / [Ne III] ratio is: 0.47775947281713343
[0.47775947281713343]


157
Galaxy:  NGC  1358
Phot table lenght: 71
No. of [Ne] data 2
Neon line 1: [Ne V] & Neon line 2: [Ne III]
Others filter names for Neon line1: no name & Neon line 2: [NeIII]
idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- -----

No. of [Ne] data 5
Neon line 1: [Ne V] & Neon line 2: [Ne III]
Others filter names for Neon line1: no name & Neon line 2: [NeIII]
idx  Observed_Passband    Flux_Density ... NED_Units       Refcode      
                               Jy      ...                              
--- -------------------- ------------- ... --------- -------------------
  0   [Ne V] 14.32 (IRS)  6630000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  1         [Ne V] 14.32  6630000000.0 ...     Jy-Hz 2011ApJS..195...17W
  2       [Ne III] 15.56  9290000000.0 ...     Jy-Hz 2011ApJS..195...17W
  3 [Ne III] 15.56 (IRS)  9290000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  4   [Ne III] (Spitzer) 10000000000.0 ...     Jy-Hz 2007ApJ...671..124D
For bibcode , 2010ApJ...709.1257T [Ne V] index is: 0 and [Ne III] index is: 3
For bibcode 2010ApJ...709.1257T [Ne V] / [Ne III] ratio is: 0.7136706135629709
For bibcode , 2011ApJS..195...17W [Ne V] index is: 1 and [Ne III] index is: 2
For bibcode 2011ApJS..195...17W [Ne V] / [Ne III] r

No. of [Ne] data 11
Neon line 1: [Ne V] & Neon line 2: [Ne III]
Others filter names for Neon line1: no name & Neon line 2: [NeIII]
idx  Observed_Passband    Flux_Density ... NED_Units       Refcode      
                               Jy      ...                              
--- -------------------- ------------- ... --------- -------------------
  0 [Ne V] 14.32 Spitzer 18100000000.0 ...     Jy-Hz 2008ApJ...674L...9D
  1    [Ne V] 14.3 (IRS) 18300000000.0 ...     Jy-Hz 2009ApJ...690.1105K
  2   [Ne V] 14.32 (IRS) 16600000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  3   [Ne V] 14.32 (IRS) 17200000000.0 ...     Jy-Hz 2010ApJ...716.1151W
  4  [Ne V] 14.3 microns 21700000000.0 ...     Jy-Hz 2007ApJ...655L..73G
  5   [Ne V] 14.32 (IRS) 16400000000.0 ...     Jy-Hz 2011ApJ...740...94D
  6 [NeIII] 15.6 microns 35300000000.0 ...     Jy-Hz 2007ApJ...655L..73G
  7  [Ne III] 15.5 (IRS) 29900000000.0 ...     Jy-Hz 2009ApJ...690.1105K
  8 [Ne III] 15.56 (IRS) 27600000000.0 ...     Jy-Hz 2010ApJ...709

No. of [Ne] data 2
Neon line 1: [Ne V] & Neon line 2: [Ne III]
Others filter names for Neon line1: no name & Neon line 2: [NeIII]
idx Observed_Passband  Flux_Density ... NED_Units       Refcode      
                            Jy      ...                              
--- ----------------- ------------- ... --------- -------------------
  0      [Ne V] 14.32 12300000000.0 ...     Jy-Hz 2011ApJS..195...17W
  1    [Ne III] 15.56 14000000000.0 ...     Jy-Hz 2011ApJS..195...17W
For bibcode 2011ApJS..195...17W , [Ne V] index is: 0 and [Ne III] index is: 1
For bibcode 2011ApJS..195...17W [Ne V] / [Ne III] ratio is: 0.8785714285714286
[0.8785714285714286]




In [12]:
passbands, countspb = np.unique(passband, return_counts=True)
for pb in range(len(passbands)): print(passbands[pb])

[Ne III] (IRS)
[Ne III] (Spitzer)
[Ne III] 15.5 (IRS)
[Ne III] 15.55 (IRS)
[Ne III] 15.55 micrn
[Ne III] 15.56
[Ne III] 15.56 (IRS)
[Ne III] 15.6 (IRS)
[Ne III] line (IRS)
[Ne V] (IRS)
[Ne V] 14.3 (IRS)
[Ne V] 14.3 Spitzer
[Ne V] 14.3 microns
[Ne V] 14.32
[Ne V] 14.32 (IRS)
[Ne V] 14.32 Spitzer
[Ne V] line (IRS)
[NeIII] 15.6 Spitzer
[NeIII] 15.6 microns


In [13]:
refcodes, countsrc = np.unique(refcode, return_counts=True)
for rc in range(len(refcodes)): print(refcodes[rc])

2004ApJS..154..178A
2006ApJ...640..204A
2006ApJ...647..161O
2006ApJ...649...79S
2007ApJ...655L..73G
2007ApJ...656..148A
2007ApJ...660..117C
2007ApJ...671..124D
2009ApJ...690.1105K
2009MNRAS.398.1165G
2010ApJ...709.1257T
2010ApJ...710..248C
2010ApJ...716.1151W
2010ApJ...725.2270P
2011A&A...533A..56P
2011ApJ...730...28P
2011ApJ...740...94D
2011ApJ...740...99D
2011ApJS..193...18W
2011ApJS..195...17W
2012ApJ...744....2A
2012ApJ...747...46P
2012ApJ...747...95G


In [14]:
print('Number of galaxies with diferent bibcode for [NeV] and [NeIII] fluxes:',not_same_bib)

Number of galaxies with diferent bibcode for [NeV] and [NeIII] fluxes: 4


## [NeIII]/[NeII]

In [33]:
passband = []
refcode = []
NeIII_NeII = Table(names=('Main_id','[NeIII]/[NeII]','[NeIII]/[NeII]_err'))

not_same_bib = 0

for id in range(len(NeIII_NeII_Sy)):
    NED_phot = Table()
    obj = NeIII_NeII_Sy['Main_Id'][id]
    print(id)
    print('Galaxy: ',obj)

    NED_phot = Table.read(path_phot_ned+obj+'_NED_phot_tables'+'.vot', format='votable')
    print('Phot table lenght:',len(NED_phot))
    
    #Put first the frequencies of the Ne line 1 and then the Ne line 2
    NeIII_NeII_Freq = [19200000000000.0,19300000000000.0,23400000000000.0]
    NeIII_NeII_inf = Neon_info(NED_phot, NeIII_NeII_Freq)
    
    NeIII_NeII_ratio,passband,refcode,not_same_bib = Ne_ratios(NeIII_NeII_inf,'[Ne III]','[Ne II]','[NeIII]',\
                                                               'no name',passband,refcode,not_same_bib)
    
    print(NeIII_NeII_ratio)
    print('\n')

0
Galaxy:  3C 234.0
Phot table lenght: 253
No. of [Ne] data 4
Neon line 1: [Ne III] & Neon line 2: [Ne II]
Others filter names for Neon line1: [NeIII] & Neon line 2: no name
idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- -------------------- ------------ ... --------- -------------------
  0 [Ne III] 15.56 (IRS) 3400000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  1 [Ne III] 15.55 micrn 8200000000.0 ...     Jy-Hz 2006ApJ...647..161O
  2   [Ne II] 12.8 (IRS) 1400000000.0 ...     Jy-Hz 2009ApJ...697..182T
  3 [Ne II] 12.81 micron  800000000.0 ...     Jy-Hz 2006ApJ...647..161O
For bibcode , 2006ApJ...647..161O [Ne III] index is: 1 and [Ne II] index is: 3
For bibcode 2006ApJ...647..161O [Ne III] / [Ne II] ratio is: 10.25
[10.25]


1
Galaxy:  2MASS J13000535+1632148
Phot table lenght: 98
No. of [Ne] data 2
Neon line 1: [Ne III] & Neon line 2: [Ne II]
Others filter names for Neon line1: [NeIII] &

Phot table lenght: 145
No. of [Ne] data 5
Neon line 1: [Ne III] & Neon line 2: [Ne II]
Others filter names for Neon line1: [NeIII] & Neon line 2: no name
idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- -------------------- ------------ ... --------- -------------------
  0       [Ne III] 15.56 9380000000.0 ...     Jy-Hz 2011ApJS..195...17W
  1 [Ne III] 15.56 (IRS) 9380000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  2        [Ne II] 12.81 9400000000.0 ...     Jy-Hz 2011ApJS..195...17W
  3  [Ne II] 12.81 (IRS) 9400000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  4   [Ne II] 12.8 (IRS) 1500000000.0 ...     Jy-Hz 2009ApJ...697..182T
For bibcode , 2010ApJ...709.1257T [Ne III] index is: 1 and [Ne II] index is: 3
For bibcode 2010ApJ...709.1257T [Ne III] / [Ne II] ratio is: 0.997872340425532
For bibcode , 2011ApJS..195...17W [Ne III] index is: 0 and [Ne II] index is: 2
For bibcode 2011ApJS..195...17W [

No. of [Ne] data 4
Neon line 1: [Ne III] & Neon line 2: [Ne II]
Others filter names for Neon line1: [NeIII] & Neon line 2: no name
idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- -------------------- ------------ ... --------- -------------------
  0 [Ne III] 15.56 (IRS) 2800000000.0 ...     Jy-Hz 2011ApJ...740...94D
  1 [Ne III] 15.56 (IRS) 2620000000.0 ...     Jy-Hz 2011ApJS..193...18W
  2  [Ne II] 12.81 (IRS) 6610000000.0 ...     Jy-Hz 2011ApJS..193...18W
  3  [Ne II] 12.81 (IRS) 6650000000.0 ...     Jy-Hz 2011ApJ...740...94D
For bibcode , 2011ApJ...740...94D [Ne III] index is: 0 and [Ne II] index is: 3
For bibcode 2011ApJ...740...94D [Ne III] / [Ne II] ratio is: 0.42105263157894735
For bibcode , 2011ApJS..193...18W [Ne III] index is: 1 and [Ne II] index is: 2
For bibcode 2011ApJS..193...18W [Ne III] / [Ne II] ratio is: 0.39636913767019666
[0.42105263157894735, 0.39636913767019666]




No. of [Ne] data 6
Neon line 1: [Ne III] & Neon line 2: [Ne II]
Others filter names for Neon line1: [NeIII] & Neon line 2: no name
idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- -------------------- ------------ ... --------- -------------------
  0 [NeIII] 15.6 Spitzer 3300000000.0 ...     Jy-Hz 2010ApJ...725.2270P
  1 [Ne III] 15.56 (IRS) 3700000000.0 ...     Jy-Hz 2010ApJ...716.1151W
  2 [Ne III] 15.56 (IRS) 3330000000.0 ...     Jy-Hz 2011ApJ...740...94D
  3 [Ne II] 12.8 Spitzer 3800000000.0 ...     Jy-Hz 2010ApJ...725.2270P
  4  [Ne II] 12.81 (IRS) 3490000000.0 ...     Jy-Hz 2011ApJ...740...94D
  5  [Ne II] 12.81 (IRS) 3690000000.0 ...     Jy-Hz 2010ApJ...716.1151W
For bibcode , 2010ApJ...716.1151W [Ne III] index is: 1 and [Ne II] index is: 5
For bibcode 2010ApJ...716.1151W [Ne III] / [Ne II] ratio is: 1.002710027100271
For bibcode , 2010ApJ...725.2270P [Ne III] index is: 0 and [Ne 

No. of [Ne] data 2
Neon line 1: [Ne III] & Neon line 2: [Ne II]
Others filter names for Neon line1: [NeIII] & Neon line 2: no name
idx Observed_Passband  Flux_Density ... NED_Units       Refcode      
                            Jy      ...                              
--- ----------------- ------------- ... --------- -------------------
  0    [Ne III] 15.56 20400000000.0 ...     Jy-Hz 2011ApJS..195...17W
  1     [Ne II] 12.81 27300000000.0 ...     Jy-Hz 2011ApJS..195...17W
For bibcode 2011ApJS..195...17W , [Ne III] index is: 0 and [Ne II] index is: 1
For bibcode 2011ApJS..195...17W [Ne III] / [Ne II] ratio is: 0.7472527472527473
[0.7472527472527473]


39
Galaxy:  Mrk 1383
Phot table lenght: 205
No. of [Ne] data 5
Neon line 1: [Ne III] & Neon line 2: [Ne II]
Others filter names for Neon line1: [NeIII] & Neon line 2: no name
idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- --------------

...

Phot table lenght: 425
No. of [Ne] data 15
Neon line 1: [Ne III] & Neon line 2: [Ne II]
Others filter names for Neon line1: [NeIII] & Neon line 2: no name
idx  Observed_Passband    Flux_Density ... NED_Units       Refcode      
                               Jy      ...                              
--- -------------------- ------------- ... --------- -------------------
  0 [Ne III] 15.56 (IRS) 17100000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  1 [NeIII] 15.6 Spitzer 16000000000.0 ...     Jy-Hz 2010ApJ...725.2270P
  2 [Ne III] 15.56 (IRS) 16200000000.0 ...     Jy-Hz 2011ApJ...740...94D
  3 [Ne III] 15.56 (IRS) 16400000000.0 ...     Jy-Hz 2010ApJ...716.1151W
  4  [Ne III] 15.5 (IRS) 16000000000.0 ...     Jy-Hz 2009MNRAS.398.1165G
  5       [Ne III] 15.56 17100000000.0 ...     Jy-Hz 2011ApJS..195...17W
  6   [Ne III] (Spitzer) 23000000000.0 ...     Jy-Hz 2007ApJ...671..124D
  7        [Ne II] 12.81 21200000000.0 ...     Jy-Hz 2011ApJS..195...17W
  8  [Ne II] 12.81 (IRS) 17300000000.0 ...

No. of [Ne] data 2
Neon line 1: [Ne III] & Neon line 2: [Ne II]
Others filter names for Neon line1: [NeIII] & Neon line 2: no name
idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- -------------------- ------------ ... --------- -------------------
  0 [Ne III] 15.56 (IRS) 6070000000.0 ...     Jy-Hz 2010ApJ...716.1151W
  1  [Ne II] 12.81 (IRS) 6740000000.0 ...     Jy-Hz 2010ApJ...716.1151W
For bibcode 2010ApJ...716.1151W , [Ne III] index is: 0 and [Ne II] index is: 1
For bibcode 2010ApJ...716.1151W [Ne III] / [Ne II] ratio is: 0.900593471810089
[0.900593471810089]


197
Galaxy:  NGC  1358
Phot table lenght: 71
No. of [Ne] data 2
Neon line 1: [Ne III] & Neon line 2: [Ne II]
Others filter names for Neon line1: [NeIII] & Neon line 2: no name
idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- -----

Phot table lenght: 100
No. of [Ne] data 2
Neon line 1: [Ne III] & Neon line 2: [Ne II]
Others filter names for Neon line1: [NeIII] & Neon line 2: no name
idx Observed_Passband Flux_Density NED_Uncertainty NED_Units       Refcode      
                           Jy                                                   
--- ----------------- ------------ --------------- --------- -------------------
  0    [Ne III] 15.56 1490000000.0                     Jy-Hz 2011ApJS..195...17W
  1     [Ne II] 12.81 6510000000.0                     Jy-Hz 2011ApJS..195...17W
For bibcode 2011ApJS..195...17W , [Ne III] index is: 0 and [Ne II] index is: 1
For bibcode 2011ApJS..195...17W [Ne III] / [Ne II] ratio is: 0.22887864823348694
[0.22887864823348694]


205
Galaxy:  ESO 103-35
Phot table lenght: 187
No. of [Ne] data 7
Neon line 1: [Ne III] & Neon line 2: [Ne II]
Others filter names for Neon line1: [NeIII] & Neon line 2: no name
idx  Observed_Passband    Flux_Density ... NED_Units       Refcode      
      

No. of [Ne] data 6
Neon line 1: [Ne III] & Neon line 2: [Ne II]
Others filter names for Neon line1: [NeIII] & Neon line 2: no name
idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- -------------------- ------------ ... --------- -------------------
  0       [Ne III] 15.56 4820000000.0 ...     Jy-Hz 2011ApJS..195...17W
  1 [Ne III] 15.56 (IRS) 4820000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  2 [Ne III] 15.56 (IRS) 4720000000.0 ...     Jy-Hz 2010ApJ...716.1151W
  3  [Ne II] 12.81 (IRS) 3200000000.0 ...     Jy-Hz 2010ApJ...716.1151W
  4        [Ne II] 12.81 3890000000.0 ...     Jy-Hz 2011ApJS..195...17W
  5  [Ne II] 12.81 (IRS) 3890000000.0 ...     Jy-Hz 2010ApJ...709.1257T
For bibcode , 2010ApJ...709.1257T [Ne III] index is: 1 and [Ne II] index is: 5
For bibcode 2010ApJ...709.1257T [Ne III] / [Ne II] ratio is: 1.2390745501285347
For bibcode , 2010ApJ...716.1151W [Ne III] index is: 2 and [Ne

No. of [Ne] data 7
Neon line 1: [Ne III] & Neon line 2: [Ne II]
Others filter names for Neon line1: [NeIII] & Neon line 2: no name
idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- -------------------- ------------ ... --------- -------------------
  0  [Ne III] 15 Spitzer 5200000000.0 ...     Jy-Hz 2010ApJ...725..605S
  1 [Ne III] 15.56 (IRS) 3860000000.0 ...     Jy-Hz 2011ApJ...740...94D
  2  [Ne III] 15.5 (IRS) 3740000000.0 ...     Jy-Hz 2009ApJ...690.1105K
  3        [Ne II] (IRS) 4110000000.0 ...     Jy-Hz 2006ApJ...649...79S
  4   [Ne II] 12.8 (IRS) 4520000000.0 ...     Jy-Hz 2009ApJ...690.1105K
  5 [Ne II] 12.8 Spitzer 5900000000.0 ...     Jy-Hz 2010ApJ...725..605S
  6  [Ne II] 12.81 (IRS) 4270000000.0 ...     Jy-Hz 2011ApJ...740...94D
For bibcode , 2009ApJ...690.1105K [Ne III] index is: 2 and [Ne II] index is: 4
For bibcode 2009ApJ...690.1105K [Ne III] / [Ne II] ratio is: 0.8274336

In [16]:
passbands, countspb = np.unique(passband, return_counts=True)
for pb in range(len(passbands)): print(passbands[pb])

[Ne III] (IRS)
[Ne III] (Spitzer)
[Ne III] 15 Spitzer
[Ne III] 15.5 (IRS)
[Ne III] 15.55 (IRS)
[Ne III] 15.55 micrn
[Ne III] 15.56
[Ne III] 15.56 (IRS)
[Ne III] 15.6 (IRS)
[Ne III] line (IRS)
[Ne II] (IRS)
[Ne II] (Spitzer)
[Ne II] 12.8 (IRS)
[Ne II] 12.8 Spitzer
[Ne II] 12.81
[Ne II] 12.81 (IRS)
[Ne II] 12.81 micron
[Ne II] 15.6 (IRS)
[Ne II] line (IRS)
[NeIII] 15.6 Spitzer


In [17]:
refcodes, countsrc = np.unique(refcode, return_counts=True)
for rc in range(len(refcodes)): print(refcodes[rc])

2004ApJS..154..178A
2004ApJS..154..242D
2006ApJ...640..204A
2006ApJ...641..795O
2006ApJ...647..161O
2007ApJ...656..148A
2007ApJ...660..117C
2007ApJ...671..124D
2007ApJ...671..323H
2008ApJ...673..193W
2008ApJ...681.1205O
2009ApJ...690.1105K
2009ApJ...693.1223S
2009ApJ...704.1159H
2009ApJ...705..885O
2009ApJS..184..230B
2009MNRAS.398.1165G
2010ApJ...709.1257T
2010ApJ...712..164H
2010ApJ...716.1151W
2010ApJ...725..605S
2010ApJ...725.2270P
2010ApJS..188..447P
2011A&A...533A..56P
2011ApJ...740...94D
2011ApJ...740...99D
2011ApJ...741...76C
2011ApJS..193...18W
2011ApJS..195...17W
2012ApJ...744....2A
2012ApJ...747...46P
2012ApJ...747...95G


In [18]:
print('Number of galaxies with diferent bibcode for [NeIII] and [NeII] fluxes:',not_same_bib)

Number of galaxies with diferent bibcode for [NeIII] and [NeII] fluxes: 2


## Notebook Info

In [19]:
%reload_ext watermark
%watermark -a "Jonhatan Bernal" -d -v -m
print('Specific Python packages')
%watermark -iv -w --packages astropy

Author: Jonhatan Bernal

Python implementation: CPython
Python version       : 3.9.12
IPython version      : 8.4.0

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.15.0-47-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

Specific Python packages
astropy: 5.1

astropy   : 5.1
matplotlib: 3.5.1
numpy     : 1.21.5

Watermark: 2.3.1

